In [12]:
#necessary imports
import random as rand
import numpy as np
from tabulate import tabulate

In [2]:
class Candidate:
    def __init__(self, bins, items, chromosome=None):
        """
        Initialise a candidate solution, as defined by the listed parameters, and immediatelly evaluate its fitness.
        :param bins: The number of bins (defined by problem)
        :param items: The items to be bin packed (defined by problem)
        :param chromosome: A candidate solution depicting item distribution in bins
        """
        
        self.bins = bins
        self.items = items
        
        # during generation of initial population, chromosomes are not specified and are instead randomly generated
        if chromosome is None:
            
            # generate an array of random ints representing bin allocation
            self.chromosome = np.random.randint(low=1, high=self.bins+1, size=len(self.items))
        else:
            self.chromosome = chromosome
            
        self.fitness = self.calculate_fitness()
        
    def calculate_fitness(self):
        """
        A fitness function is defined here as the difference in weight of the heaviest bin and the lightest bin.
        :returns: fitness as defined
        """
        
        bin_weights = {}
        
        # allocate item weights to bins using the chromosome (bin allocation)
        for idx, allocation in enumerate(self.chromosome):
            
            if allocation not in bin_weights:
                bin_weights[allocation] = 0
                
            # for each gene/allocation, add weight to appropriate bag count
            bin_weights[allocation] += self.items[idx]
        
        
        # calculate fitness and return
        return max(bin_weights.values()) - min(bin_weights.values())
    
    def m_gene_mutation(self, m):
        """
        Perform m-gene mutation on candidate solution and amend it's chromosome as appropriate
        :param m: Number of times to perform the mutation
        """
        # perform single gene mutation m times
        for mutation in range(m):
            choose_gene = rand.randint(0, len(self.chromosome)-1)
            self.chromosome[choose_gene] = rand.randint(1,self.bins)
        
        
        
        
def single_point_crossover(parent_a,parent_b):
    """
    Perform single point crossover on parents and return resultant children
    :param parent_a: A parent chromosome
    :param parent_b: A parent chromosome
    
    :returns child_a: child chromosome
    :returns child_b: child chromosome
    """
    
    # choose a random index for crossover point
    crossover_point = rand.randint(1, len(parent_a)-2)
    
    # split parents down this crossover point
    parent_a_start, parent_a_end = parent_a[:crossover_point], parent_a[crossover_point:]
    parent_b_start, parent_b_end = parent_b[:crossover_point], parent_b[crossover_point:]
    
    # combine parent pieces to form children
    child_a = np.hstack((parent_a_start, parent_b_end))
    child_b = np.hstack((parent_a_end, parent_b_start))
    
    return child_a, child_b


def binary_tournament(population):
    """
    Perform binary tornament without replacement on population
    :param population: List of candidate solution objects
    """
    # select 2 random candidates
    # here replace is set to false to avoid a tournament between the same object
    candidates = np.random.choice(population, 2, replace=False)
    
    # return the candidate of smallest fitness value
    return min(candidates, key=lambda candidate: candidate.fitness)

def weakest_replacement(population, candidate):
    """
    Find weakest current population memeber and compare them to the candidate. 
    Replace with candidate if candidate has a better fitness function.
    
    :param population:List of candidate solution objects
    :param candidate: A candidate solution object
    """
    
    # Find the worst solution
    worst_solution = max(population, key=lambda solution: solution.fitness)
    
    if candidate.fitness <= worst_solution.fitness:
        population.remove(worst_solution)
        population.append(candidate)
        
    return population


def solve_bin(bins,items,m,population,crossover=True):
    """
    Run the bin packing EA, with the specified parameters.
    :param bins: The number of bins as defined in problem
    :param items: The list of items as defined by problem
    :param m: The mutation coefficient
    :param population: The size of the population
    :param crossover: Whether to include crossover
    """
    
    # initiate iterations
    itr = 0

    #generate random population
    pop = [Candidate(bins=bins,items=items) for _ in range(population)]
    
    # as candidates are evaluated upon iteration increment iterations for each of the above candiates
    itr += population
    
    while itr < 10000:
        
        # binary tournament to find parents
        parent_a, parent_b = binary_tournament(pop), binary_tournament(pop)
        
        if crossover:           
            # children chromosomes created via single point crossover
            child_a, child_b = single_point_crossover(parent_a.chromosome,parent_b.chromosome)
            
        else:
            #no crossover so children just inherit parents' chromosomes
            child_a, child_b = parent_a.chromosome[:], parent_b.chromosome[:]
            
            
        #create child objects using the above chromosomes
        child_a = Candidate(bins=bins,items=items, chromosome = child_a)
        child_b = Candidate(bins=bins,items=items, chromosome = child_b)
        
        #check for mutation coefficient (no need to call function if mutation set to 0)
        if m != 0:
            child_a.m_gene_mutation(m)
            child_b.m_gene_mutation(m)
        
            # evaluate children
            child_a.fitness= child_a.calculate_fitness()
            child_b.fitness = child_b.calculate_fitness()
        
        #incrmeent evaluation counter for each child being evaluated
        itr += 2 
        
        # update population using weakest replacement
        pop = weakest_replacement(pop, child_a)
        pop = weakest_replacement(pop, child_b)
        
    # return the candidate with minimal fitness function from the entire population
    # this is the top candidate
    return min(pop, key=lambda solution: solution.fitness).fitness

In [3]:
# test runner
def run_trial(trial_name,bins,items,mutation,crossover,population,n_trials):
    """
    Run Tests with warying parameters.
    """
    
    results = []
    
    for n in range(n_trials):
        print("Running Trial", trial_name, ": ", n+1 , "/", n_trials)
        # set random seed
        rand.seed(n)
        np.random.seed(n)
        
        result = solve_bin(bins,items,mutation,population,crossover=crossover)
        results.append(result)
    
    print()
    return results
    
    

In [4]:
bpp1 = list(map(lambda x: 2*x, list(range(1, 501))))
bpp2 = list(map(lambda x: 2*x**2 , list(range(1, 501))))

bpp1_results = [[],[],[],[],[],[]]
bpp2_results = [[],[],[],[],[],[]]

bpp1_results[0] = run_trial("Crossover and M1 (Population 10)", 10, bpp1, 1, True, 10, 5)
bpp1_results[1] = run_trial("Crossover and M1 (Population 100)", 10, bpp1, 1, True, 100, 5)
bpp1_results[2] = run_trial("Crossover and M5 (Population 10)", 10, bpp1, 5, True, 10, 5)
bpp1_results[3] = run_trial("Crossover and M5 (Population 100)", 10, bpp1, 5, True, 100, 5)
bpp1_results[4] = run_trial("M1 (Population 10)", 10, bpp1, 5, False, 10, 5)
bpp1_results[5] = run_trial("Crossover and M0 (Population 10)", 10, bpp1, 0, True, 10, 5)

bpp2_results[0] = run_trial("Crossover and M1 (Population 10)", 100, bpp2, 1, True, 10, 5)
bpp2_results[1] = run_trial("Crossover and M1 (Population 100)", 100, bpp2, 1, True, 100, 5)
bpp2_results[2] = run_trial("Crossover and M5 (Population 10)", 100, bpp2, 5, True, 10, 5)
bpp2_results[3] = run_trial("Crossover and M5 (Population 100)", 100, bpp2, 5, True, 100, 5)
bpp2_results[4] = run_trial("M1 (Population 10)", 100, bpp2, 5, False, 10, 5)
bpp2_results[5] = run_trial("Crossover and M0 (Population 10)", 100, bpp2, 0, True, 10, 5)


Running Trial Crossover and M1 (Population 10) :  1 / 5
Running Trial Crossover and M1 (Population 10) :  2 / 5
Running Trial Crossover and M1 (Population 10) :  3 / 5
Running Trial Crossover and M1 (Population 10) :  4 / 5
Running Trial Crossover and M1 (Population 10) :  5 / 5

Running Trial Crossover and M1 (Population 100) :  1 / 5
Running Trial Crossover and M1 (Population 100) :  2 / 5
Running Trial Crossover and M1 (Population 100) :  3 / 5
Running Trial Crossover and M1 (Population 100) :  4 / 5
Running Trial Crossover and M1 (Population 100) :  5 / 5

Running Trial Crossover and M5 (Population 10) :  1 / 5
Running Trial Crossover and M5 (Population 10) :  2 / 5
Running Trial Crossover and M5 (Population 10) :  3 / 5
Running Trial Crossover and M5 (Population 10) :  4 / 5
Running Trial Crossover and M5 (Population 10) :  5 / 5

Running Trial Crossover and M5 (Population 100) :  1 / 5
Running Trial Crossover and M5 (Population 100) :  2 / 5
Running Trial Crossover and M5 (Popula

In [5]:
# print average fitness for each trial(bpp1)
for result in bpp1_results:
    print("Average fitness for bpp1:", (sum(result)/5) )

Average fitness for bpp1: 51.6
Average fitness for bpp1: 186.0
Average fitness for bpp1: 437.6
Average fitness for bpp1: 773.6
Average fitness for bpp1: 3641.6
Average fitness for bpp1: 3640.8


In [8]:
# print average fitness for each trial(bpp2)
for result in bpp2_results:
    print("Average fitness for bpp2:", (sum(result)/5) )

Average fitness for bpp2: 674310.8
Average fitness for bpp2: 1077790.8
Average fitness for bpp2: 632706.8
Average fitness for bpp2: 967680.8
Average fitness for bpp2: 1584986.8
Average fitness for bpp2: 1685891.6


In [16]:
# comparison of different population and small mutation
header = ["Trial 1","Trial 2","Trial 3","Trial 4","Trial 5" ]
table = ([["Population 10 (M1)", *bpp1_results[0]],["Population 100 (M1)", *bpp1_results[1]]])
print(tabulate(table, headers=header))

                       Trial 1    Trial 2    Trial 3    Trial 4    Trial 5
-------------------  ---------  ---------  ---------  ---------  ---------
Population 10 (M1)          46         70         44         36         62
Population 100 (M1)        238        196        164        176        156


In [22]:
# comparison of different population and large mutation
header = ["Trial 1","Trial 2","Trial 3","Trial 4","Trial 5" ]
table = ([["Population 10 (M5)", *bpp1_results[2]],["Population 100 (M5)", *bpp1_results[3]]])
print(tabulate(table, headers=header))

                       Trial 1    Trial 2    Trial 3    Trial 4    Trial 5
-------------------  ---------  ---------  ---------  ---------  ---------
Population 10 (M5)         372        566        436        282        532
Population 100 (M5)        646        844        812        766        800


In [18]:
# comparing different mutation on small population
header = ["Trial 1","Trial 2","Trial 3","Trial 4","Trial 5" ]
table = ([["Population 10 (M1)", *bpp1_results[0]],["Population 10 (M5)", *bpp1_results[2]]])
print(tabulate(table, headers=header))

                      Trial 1    Trial 2    Trial 3    Trial 4    Trial 5
------------------  ---------  ---------  ---------  ---------  ---------
Population 10 (M1)         46         70         44         36         62
Population 10 (M5)        372        566        436        282        532


In [19]:
# comparing different mutation on large population
header = ["Trial 1","Trial 2","Trial 3","Trial 4","Trial 5" ]
table = ([["Population 100 (M1)", *bpp1_results[1]],["Population 100 (M5)", *bpp1_results[3]]])
print(tabulate(table, headers=header))

                      Trial 1    Trial 2    Trial 3    Trial 4    Trial 5
------------------  ---------  ---------  ---------  ---------  ---------
Population 10 (M1)        238        196        164        176        156
Population 10 (M5)        646        844        812        766        800


In [24]:
#comparing both additions, lack of crossover, lack of mutation
header = ["Trial 1","Trial 2","Trial 3","Trial 4","Trial 5" ]
table = ([["Both Present", *bpp1_results[0]],["No Crossover", *bpp1_results[4]],["No Mutation", *bpp1_results[5]]])
print(tabulate(table, headers=header))

                Trial 1    Trial 2    Trial 3    Trial 4    Trial 5
------------  ---------  ---------  ---------  ---------  ---------
Both Present         46         70         44         36         62
No Crossover       2800       4200       3618       4256       3334
No Mutation        1878       2326       6514       3694       3792


In [35]:
#all results
header = ["Problem 1","Trial 1","Trial 2","Trial 3","Trial 4","Trial 5", "Average" ]
table1 = ([["Crossover and M1 (Population 10)", *bpp1_results[0], sum(bpp1_results[0])/5],
          ["Crossover and M1 (Population 100)", *bpp1_results[1],sum(bpp1_results[1])/5],
          ["Crossover and M5 (Population 10)", *bpp1_results[2],sum(bpp1_results[2])/5],
          ["Crossover and M5 (Population 100)", *bpp1_results[3],sum(bpp1_results[3])/5],
          ["M1 (Population 10)", *bpp1_results[4],sum(bpp1_results[4])/5],
          ["Crossover and M0 (Population 10)", *bpp1_results[5],sum(bpp1_results[5])/5],
          ])
print(tabulate(table1, headers=header))
print("--------------------------------------------------------------------------------------------------------------------")
           
header = ["Problem 2","Trial 1","Trial 2","Trial 3","Trial 4","Trial 5", "Average" ]
table2 = ([["Crossover and M1 (Population 10)", *bpp2_results[0],sum(bpp2_results[0])/5],
          ["Crossover and M1 (Population 100)", *bpp2_results[1],sum(bpp2_results[1])/5],
          ["Crossover and M5 (Population 10)", *bpp2_results[2],sum(bpp2_results[2])/5],
          ["Crossover and M5 (Population 100)", *bpp2_results[3],sum(bpp2_results[3])/5],
          ["M1 (Population 10)", *bpp2_results[4],sum(bpp2_results[4])/5],
          ["Crossover and M0 (Population 10)", *bpp2_results[5],sum(bpp2_results[5])/5],
          ])
print(tabulate(table2, headers=header))

Problem 1                            Trial 1    Trial 2    Trial 3    Trial 4    Trial 5    Average
---------------------------------  ---------  ---------  ---------  ---------  ---------  ---------
Crossover and M1 (Population 10)          46         70         44         36         62       51.6
Crossover and M1 (Population 100)        238        196        164        176        156      186
Crossover and M5 (Population 10)         372        566        436        282        532      437.6
Crossover and M5 (Population 100)        646        844        812        766        800      773.6
M1 (Population 10)                      2800       4200       3618       4256       3334     3641.6
Crossover and M0 (Population 10)        1878       2326       6514       3694       3792     3640.8
--------------------------------------------------------------------------------------------------------------------
Problem 2                            Trial 1    Trial 2    Trial 3    Trial 4    Tria